In [3]:
!pip install git+http://github.com/afnan47/cuda.git

  Cloning http://github.com/afnan47/cuda.git to /tmp/pip-req-build-x3pol_65
  Running command git clone --filter=blob:none --quiet http://github.com/afnan47/cuda.git /tmp/pip-req-build-x3pol_65
  Resolved http://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=c867e9a796c4ca96d7fcd1c095278bd87333cca43549fbef533deb5c186c2baf
  Stored in directory: /tmp/pip-ephem-wheel-cache-oywncl9u/wheels/e1/4a/af/7b73bdd6e26691feafcd343785df2e333d634293016011532c
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [24]:
%%cu
#include <iostream>
using namespace std;

__global__ void CudaMultiplication(int* x, int* y, int* z, int N1, int N2, int M1, int M2){
      int row = blockIdx.x * blockDim.x + threadIdx.x;
      int col = blockIdx.y * blockDim.y + threadIdx.y;
      int val = 0;


      for(int k=0; k<N2; k++){
          val += x[row*N2+k]*y[k*M2+col] ;
      }

      z[row*M2+col]=val;
}

int main(){

    int N1 = 3;
    int N2 = 4;
    int M1 = 4;
    int M2 = 1;

    int *x = (int *)malloc(sizeof(int)*N1*N2);
    int *y = (int *)malloc(sizeof(int)*M1*M2);
    int *z = (int *)malloc(sizeof(int)*N1*M2);

    for(int i=0;i<N1;i++){
      for(int j=0;j<N2;j++) {
          x[i*N2+j]=1;
      }
    }

    for(int i=0;i<M1;i++){
      for(int j=0;j<M2;j++) {
          y[i*M2+j]=1;
      }
    }

    int *a , *b , *c;
    cudaMalloc(&a , sizeof(int)*N1*N2);
    cudaMalloc(&b , sizeof(int)*M1*M2);
    cudaMalloc(&c , sizeof(int)*N1*M2);

    cudaMemcpy(a, x, sizeof(int)*N1*N2, cudaMemcpyHostToDevice);
    cudaMemcpy(b, y, sizeof(int)*M1*M2, cudaMemcpyHostToDevice);

    dim3 th(N1,M2);
    CudaMultiplication<<<1, th>>>(a, b, c, N1, N2, M1, M2);

    cudaMemcpy(z, c, sizeof(int)*N1*M2, cudaMemcpyDeviceToHost);

    for(int i=0;i<N1;i++){
        for(int j=0;j<M2;j++){
            cout<<z[i*M2+j]<<" ";
        }
        cout<<endl;
    }
    return 0;
}

4 
4 
4 



In [26]:
%%cu
#include <iostream>
using namespace std;

__global__ void CudaAddition(int* x, int* y, int* z, int N){
      int index = blockIdx.x * blockDim.x + threadIdx.x;

      if (index < N) {
          z[index] = x[index]+y[index];
      }
}

int main(){
    int N = 3;

    int *x = (int *)malloc(sizeof(int)*N);
    int *y = (int *)malloc(sizeof(int)*N);
    int *z = (int *)malloc(sizeof(int)*N);


    for(int i=0;i<N;i++){
          x[i]=1;
          y[i]=1;
    }


    int *a , *b , *c;
    cudaMalloc(&a , sizeof(int)*N);
    cudaMalloc(&b , sizeof(int)*N);
    cudaMalloc(&c , sizeof(int)*N);

    cudaMemcpy(a, x, sizeof(int)*N, cudaMemcpyHostToDevice);
    cudaMemcpy(b, y, sizeof(int)*N, cudaMemcpyHostToDevice);

    dim3 th(N);
    CudaAddition<<<1, th>>>(a, b, c, N);

    cudaMemcpy(z, c, sizeof(int)*N, cudaMemcpyDeviceToHost);

    for(int i=0;i<N;i++){
        cout<<z[i]<<" ";
    }
    return 0;
}

2 2 2 
